In [16]:
import pandas as pd
import seaborn as sns
import janitor

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
# Configuración del aspecto general de las gráficas
%matplotlib inline

sns.set_theme(
    rc= {
        'figure.figsize': (8, 6)
    }
)

sns.set_style("whitegrid")

## El problema de trabajar con valores faltantes

Comencemos con un ejemplo sobre cómo trabajar con valores faltantes nos puede traer problemas. Para esto vamos a acceder a un conjunto de datos que nos provee el submódulo `datasets` de `statsmodels.api` los cuales tienen origen desde conjuntos de datos de `R`, los cuales son meramente de prueba.

### Conjunto de datos Air Quality

Medidas de Calidad del Aire en Nueva York

**Descripción**

El conjunto de datos contiene mediciones diarias de la calidad del aire en Nueva York, recopiladas desde mayo hasta septiembre de 1973.

**Formato**

El conjunto de datos es un DataFrame con 153 observaciones sobre 6 variables:

**Variables contenidas en el conjunto de datos**

- **Ozone**: Ozono (ppb)
- **Solar.R**: Radiación solar (lang)
- **Wind**: Velocidad del viento (mph)
- **Temp**: Temperatura (grados Fahrenheit)
- **Month**: Mes (1-12)
- **Day**: Día del mes (1-31)

**Detalles**

Las lecturas diarias de los valores de calidad del aire fueron tomadas desde el 1 de mayo de 1973 (un martes) hasta el 30 de septiembre de 1973. Las variables medidas incluyen:

- **Ozone**: Media de ozono en partes por billón (ppb) entre las 13:00 y 15:00 horas en Roosevelt Island.
- **Solar.R**: Radiación solar en Langleys en el rango de frecuencias de 4000–7700 Angstroms entre las 08:00 y 12:00 horas en Central Park.
- **Wind**: Velocidad media del viento en millas por hora a las 07:00 y 10:00 horas en el Aeropuerto LaGuardia.
- **Temp**: Temperatura máxima diaria en grados Fahrenheit en el Aeropuerto LaGuardia.

**Fuente**

Los datos fueron obtenidos del Departamento de Conservación del Estado de Nueva York (datos de ozono) y del Servicio Meteorológico Nacional (datos meteorológicos).

**Referencias**

Chambers, J. M., Cleveland, W. S., Kleiner, B. and Tukey, P. A. (1983). Graphical Methods for Data Analysis. Belmont, CA: Wadsworth.

Esta información se encuentra en el atributo `__doc__` del objeto Dataset obtenido de la función `get_rdataset`.

In [28]:
airquality_df = (
    # El método nos devuelve un objeto Dataset 
    sm.datasets.get_rdataset('airquality')
    # Accedemos al DataFrame
    .data
    # Convertimos la estructura de los nombres a Snake Case
    .clean_names(
        case_type= 'snake'
    )
    # Añadimos la columna 'year' con el valor [1973] por defecto
    .add_column('year', 1973)
    # Con ello creamos una columna para contener la fecha completa en formato [datetime]
    .assign(
        date= lambda df: pd.to_datetime(df[['year', 'month', 'day']])
    )
    # Ordenamos las observaciones por fecha
    .sort_values(by= 'date')
    # Convertimos la columna de fecha a índice
    .set_index('date')
)

airquality_df.head(3)

c:\Users\Pável Hernández\Documents\platzi\missing_data_inputation\env\Lib\site-packages\pandas_flavor\register.py:157: FutureWarning: This function will be deprecated in a 1.x release. Please use `pd.DataFrame.assign` instead.
  return method(self._obj, *args, **kwargs)


,ozone,solar_r,wind,temp,month,day,year
date,,,,,,,
1973-05-01,41.0,190.0,7.4,67,5,1,1973
1973-05-02,36.0,118.0,8.0,72,5,2,1973
1973-05-03,12.0,149.0,12.6,74,5,3,1973


Imaginemos que utilizaremos este conjunto de datos y queremos ajustar algún modelo de regresión lineal. Utilizamos `statsmodels` para esto. Nuestra intención es intentar predecir la temperatura a partir de los valores el ozono, variable que contiene datos faltantes.

In [29]:
(
    # Usamos [ols] de statsmodels.formula
    smf.ols(
        # Obtenemos la temperatura a partir del ozono
        formula= 'temp ~ ozone',
        # Proporcionamos nuestro DataFrame como argumento de datos
        data= airquality_df
    )
    # Realizamos el ajuste del modelo
    .fit()
    # Obtenemos el resumen de nuestro modelo
    .summary()
    # Accedemos a la tabla de nuestro interés
    .tables[0]
)

Dep. Variable:,temp,R-squared:,0.488
Model:,OLS,Adj. R-squared:,0.483
Method:,Least Squares,F-statistic:,108.5
Date:,"Mon, 17 Jun 2024",Prob (F-statistic):,2.93e-18
Time:,20:29:22,Log-Likelihood:,-386.27
No. Observations:,116,AIC:,776.5
Df Residuals:,114,BIC:,782.1
Df Model:,1,,
Covariance Type:,nonrobust,,


Podemos ver que en nuestro resultado tenemos 116 observaciones analizadas.

¿Qué pasa si ahora consideramos otra variable además del ozono?

In [8]:
(
    smf.ols(
        formula= 'temp ~ ozone + solar_r',
        data= airquality_df
    )
    .fit()
    .summary()
    .tables[0]
)

Dep. Variable:,temp,R-squared:,0.491
Model:,OLS,Adj. R-squared:,0.481
Method:,Least Squares,F-statistic:,52.07
Date:,"Thu, 13 Jun 2024",Prob (F-statistic):,1.47e-16
Time:,20:29:27,Log-Likelihood:,-369.78
No. Observations:,111,AIC:,745.6
Df Residuals:,108,BIC:,753.7
Df Model:,2,,
Covariance Type:,nonrobust,,


Podemos observar que ahora sólo hay 111 observaciones analizadas. No podemos hacer una comparación de este modelo con el anterior ya que desde un inicio las observaciones ingresadas al modelo no son las mismas, por ende, los resultados serán distintos.

Esto sucede porque hemos perdido datos al ingresar esta segunda variable ya que ésta también contiene valores faltantes en algunas observaciones.

Por defecto, se están descartando estas observaciones incompletas por el modelo. Esto significa un gran problema y debemos estar atentos a ello. Por esto, se requiere abordar el problema para encontrar una solución adecuada y que nos ayude a realizar el ajuste de nuestros modelos de una forma deseada y correcta.